Zookeeper包含两种安装模式

- 单机模式（用于开发和测试，没有什么意义）
- 复制模式（Zookeeper集群）

这里有三台主机：


| 主机名称 | 主机IP | 账号 |
| --- | --- | --- |
| tiny01 | 192.168.199.112 | grid |
| tiny02 | 192.168.199.113 | grid |
| tiny03 | 192.168.199.114 | grid |

# 1. 单机模式
在tiny01上安装。

## 1.1 安装Java
略...

## 1.2 下载安装包：
运行

```
$ wget https://mirrors.tuna.tsinghua.edu.cn/apache/zookeeper/zookeeper-3.4.13/zookeeper-3.4.13.tar.gz
```

下载安装包，然后运行

```
$ tar -zxvf zookeeper-3.4.13.tar.gz
```
解压安装包

## 1.3 配置全局变量
在`~/.bashrc`下面追加Zookeeper的全局变量

```
export ZOOKEEPER_HOME=~/zookeeper/zookeeper-3.4.13
export PATH=$PATH:$ZOOKEEPER_HOME/bin
```

然后编译`~/.bashrc`

```
$ source ~/.bashrc
```

## 1.4 创建配置文件

```
$ cp conf/zoo_sample.cfg conf/zoo.cfg
$ vim conf/zoo.cfg
```

设置`dataDir`路径

```
dataDir=/home/grid/zookeeper/zookeeper-3.4.13/data
```
然后创建该目录

```
$ mkdir /home/grid/zookeeper/zookeeper-3.4.13/data
```

## 1.5 启动服务

```
$ zkServer.sh start
ZooKeeper JMX enabled by default
Using config: /home/grid/zookeeper/zookeeper-3.4.13/bin/../conf/zoo.cfg
Starting zookeeper ... STARTED
```

# 2. 复制模式
Zookeeper单机模式只会存在于开发环境或测试环境中。如果想要使用它需要部署在多台主机上，使用复制模式。
## 2.1 修改配置文件
复制模式在上面的基础上修改配置文件`conf/zoo.cfg`，添加服务器配置：

```
tickTime=2000
initLimit=5
syncLimit=2
server.1=tiny01:2888:3888
server.2=tiny02:2888:3888
server.3=tiny03:2888:3888
```

**1. tickTime**

Zookeeper的客户端保证会话有效时ping的间隔时间，后面会介绍。

**2. initLimit**

设置领导者与所有跟随者（后面会介绍）连接同步时间的范围，上面设置的是`tickTime`的5倍，如果这段时间内，未能和半数以上的主机完成同步，领导者会放弃他的领导地位，重新选举出一个领导者。 

**3. syncLimit**

设置一个跟随者同领导者的同步时间，如果该事件内跟随者未完成同步，则自己会重启，会将关联到他的客户端连接到其他的跟随者上，上面设置的是`tickTime`的2倍

**3. server**

Zookpeer集群的节点情况设置。他的格式为：

```
server.[serverID]=[hostname].[FollowPort].[ElectionPort]
```

- 第一个字段固定为server；
- 第二个字段为服务器ID；
- 第三个字段为服务器hostname；
- 第四个字段为跟随者连接领导者的端口，一般为2888；
- 第五个字段用于选举阶段连接其他服务器，一般为3888；

## 2.2 设置服务器ID
在`dataDir`目录下创建一个`myid`的纯文本文件，用于设置服务器ID，例如：

```
$  vim $ZOOKEEPER_HOME/data/myid
1
```

## 2.3 将Zookeeper发送到其他主机上。

```
$ scp -r zookeeper grid@tiny02:/home/grid/
$ scp -r zookeeper grid@tiny03:/home/grid/
```

## 2.4 配置环境变量
由于这三台主机配置相同，所以直接使用scp复制了。

```
$ scp ~/.bashrc grid@tiny02:/home/grid/.bashrc
$ scp ~/.bashrc grid@tiny03:/home/grid/.bashrc
```

## 2.4 依次连接另外的主机，修改myid并其中zookeeper

```
$ ssh grid@tiny02 "echo 2 > /home/grid/zookeeper/zookeeper-3.4.13/data/myid"
$ ssh grid@tiny03 "echo 3 > /home/grid/zookeeper/zookeeper-3.4.13/data/myid"
$ ssh grid@tiny02 "source /etc/profile;source /home/grid/.bashrc;/home/grid/zookeeper/zookeeper-3.4.13/bin/zkServer.sh start"
$ ssh grid@tiny03 "source /etc/profile;source /home/grid/.bashrc;/home/grid/zookeeper/zookeeper-3.4.13/bin/zkServer.sh start"
```

<font color="red">注意：启动zookeeper需要JAVA_HOME环境变量，所以需要source一下`/etc/profile`或者`~/.bashrc`。因为通过SSH执行的命令是不会自动加载环境变量的。</font>
## 2.5 关闭所有主机的防火墙
需要三个端口是允许防火墙通过的2181/2888/3888，考虑到一般这样的集群我们都是在内网中，所以不需要防火墙，所以关闭掉所有主机的防火墙，关闭方式为切换到主机的root用户运行：

```
$ service iptables stop
```

# 3. 测试是否启动成功
运行：

```
$ echo ruok | nc tiny01 2181
$ echo ruok | nc tiny02 2181
$ echo ruok | nc tiny03 2181
```

如果返回的是

    imok

就是启动成功

# 4. 关于错误 This ZooKeeper instance is not currently serving requests
启动可能会出现下面的错误

    This ZooKeeper instance is not currently serving requests
    
这是由于Zookeeper机器正在运行的小于总数的半数而造成的，在我这里是由于没有关闭防火墙的原因。运行：
```
$ telnet tiny02 2181
```
会出现下面的提示：

    telnet: connect to address x.x.x.x: No route to host

然后关闭防火墙就可以运行了。